### Reference - https://huggingface.co/learn/cookbook/rag_zephyr_langchain

In [1]:
!pip install -q torch transformers accelerate bitsandbytes sentence-transformers faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 31.4 MB/s eta 0:00:00


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install -q langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.7 MB/s eta 0:00:00


### Prepare Data

In [1]:
from getpass import getpass
ACCESS_TOKEN = getpass("YOUR_GITHUB_PERSONAL_TOKEN")

YOUR_GITHUB_PERSONAL_TOKEN··········


In [2]:
from langchain.document_loaders import GitHubIssuesLoader
loader = GitHubIssuesLoader(repo="huggingface/peft", access_token=ACCESS_TOKEN, include_prs=False, state="all")
docs = loader.load()

What are other type of document_loaders??

In [3]:
print(docs[0]) # {page_content, metadata}
print(type(docs[0]))

page_content='### System Info

请扫微信二维码加群，如果群失效，可以添加我微信加入：yx116169
![20240725-111054](https://github.com/user-attachments/assets/cb500a4a-8008-40f8-bbf6-38472b2dca5b)

### Who can help?

1

### Information

- [X] The official example scripts
- [X] My own modified scripts

### Tasks

- [X] An officially supported task in the `examples` folder
- [X] My own task or dataset (give details below)

### Reproduction

1

### Expected behavior

1' metadata={'url': 'https://github.com/huggingface/peft/issues/1953', 'title': '拉了一个多模态大模型技术交流群，大家可以加入进来进行技术交流', 'creator': 'feihuamantian', 'created_at': '2024-07-25T04:13:58Z', 'comments': 0, 'state': 'open', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 1953, 'is_pull_request': False}
<class 'langchain_core.documents.base.Document'>


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)

In [5]:
print(type(chunked_docs[0]))
print(chunked_docs[0])

<class 'langchain_core.documents.base.Document'>
page_content='### System Info

请扫微信二维码加群，如果群失效，可以添加我微信加入：yx116169
![20240725-111054](https://github.com/user-attachments/assets/cb500a4a-8008-40f8-bbf6-38472b2dca5b)

### Who can help?

1

### Information

- [X] The official example scripts
- [X] My own modified scripts

### Tasks

- [X] An officially supported task in the `examples` folder
- [X] My own task or dataset (give details below)

### Reproduction

1

### Expected behavior

1' metadata={'url': 'https://github.com/huggingface/peft/issues/1953', 'title': '拉了一个多模态大模型技术交流群，大家可以加入进来进行技术交流', 'creator': 'feihuamantian', 'created_at': '2024-07-25T04:13:58Z', 'comments': 0, 'state': 'open', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 1953, 'is_pull_request': False}


### Create embeddings, vector DB and retriever

In [6]:
from langchain.vectorstores import FAISS # Fb AI Similary Search --> Read more here - https://ai.meta.com/tools/faiss/
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")) # Explore more embeddings here (Massive Text Embedding benchmark) - https://huggingface.co/spaces/mteb/leaderboard

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

In [7]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### Lets load a model to be used - Try quantized llama 3.1?

In [22]:
#!pip install --upgrade transformers
!pip install -q ipywidgets

In [9]:
from huggingface_hub import login
login()

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [11]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type= "nf4"
                                         )

llama_model = AutoModelForCausalLM.from_pretrained(
	model_name, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### Setup LLM chain

In [12]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

text_generation_pipeline = pipeline(
    model=llama_model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [13]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [23]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = {"context": db.as_retriever(), "question": RunnablePassthrough()} | llm_chain

### Comparing results without and with context

In [15]:
question = "How do you combine multiple adapters?"

In [20]:
print(llm_chain.invoke({"context": "", "question": question}))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



<|system|>
Answer the question based on your knowledge. Use the following context to help:



</s>
<|user|>
How do you combine multiple adapters?
</s>
<|assistant|>

  There are several ways to combine multiple adapters, depending on the specific requirements and constraints of the project. Here are some common methods:

1. **Serial Connection**: Connect one adapter to another using a serial connection (e.g., USB-to-USB). This method is useful when working with devices that have limited ports or require a specific interface.
2. **Hub or Switch**: Use a hub or switch to connect multiple adapters together. This setup allows for simultaneous communication between all connected devices.
3. **Network Bridging**: Combine multiple adapters by creating a network bridge. This involves connecting two or more adapters to a central device, which acts as a bridge, allowing data to be transmitted between them.
4. **Virtual Adapters**: Utilize virtual adapters, such as software-defined networking (S

In [25]:
print(rag_chain.invoke(question))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



<|system|>
Answer the question based on your knowledge. Use the following context to help:

[Document(metadata={'url': 'https://github.com/huggingface/peft/issues/1802', 'title': 'Issues when switching between multiple adapters LoRAs ', 'creator': 'JhonDan1999', 'created_at': '2024-05-26T19:18:13Z', 'comments': 8, 'state': 'closed', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 1802, 'is_pull_request': False}, page_content='The documentation does not mention the need to perform a merge when switching adapters. Additionally, the methods add_adapter, set_adapter, and enable_adapters do not appear to work\r\n\r\nPlease provide clarification on how to correctly switch between adapters'), Document(metadata={'url': 'https://github.com/huggingface/peft/issues/1045', 'title': 'add_weighted_adapter() is unusable, throws error: "Invalid type <class \'list\'> found in target_modules"', 'creator': 'Vectorrent', 'created_at': '2023-10-22T21:42:32Z', 'comments': 6, '